# 3.4 Seguridad Básica

- Objetivo: principios de mínimos privilegios y roles.
- Prerrequisitos: usuarios y esquemas definidos.
- Ejercicios: auditoría de permisos (conceptual).
- Reto: plan de acceso por roles.
- Errores comunes: privilegios excesivos, falta de revisión periódica.

🟢 Ejercicio: Identificar permisos mínimos para usuario analista (solo SELECT en tablas de reportes).

🟠 Ejercicio: Diseñar roles (lectura, escritura, admin) con permisos granulares.

🔴 Reto: Auditoría trimestral de permisos con eliminación de accesos no utilizados.